# Using Applications

To run different protein design applications, we first have to import everything we need. In this case, we want to test the application ligandmpnn.

In [ ]:
from protflow.poses import Poses
from protflow.tools.ligandmpnn import LigandMPNN
from protflow.jobstarters import SbatchArrayJobstarter, LocalJobStarter

First, we define our jobstarters. We want to test if we can run LigandMPNN on GPU or cpu using the SLURM workload manager as well as run it locally. If SLURM is not installed on your machine, you can skip the parts mentioning it.

In [ ]:
slurm_gpu_jobstarter = SbatchArrayJobstarter(max_cores=10, gpus=1)
slurm_cpu_jobstarter = SbatchArrayJobstarter(max_cores=10, gpus=False)

In [ ]:
local_jobstarter = LocalJobStarter(max_cores=1)

Next, we have to load our poses. We set the local_jobstarter as default jobstarter.

In [ ]:
my_poses = Poses(poses='data/input_pdbs/', glob_suffix='*pdb', work_dir='runners_example', storage_format='csv', jobstarter=local_jobstarter)

To run ligandmpnn, we have to create a runner. Make sure the path to the LigandMPNN script and python path are set in protflow/config.py! The lines there should look like:

#ligandmpnn.py

LIGANDMPNN_SCRIPT_PATH = "/home/user/LigandMPNN/run.py"

LIGANDMPNN_PYTHON_PATH = "/home/user/anaconda3/envs/ligandmpnn_env/bin/python3"

You can set it also when creating the runner, but it is recommended to set it in the config if you want to run it again.

If running this notebook on a cluster, be sure that is also opened from there (e.g. with VS code installed on the cluster)! Otherwise, the /home/ directories won't match since it will look for the files on your local machine and not on the cluster!

In [ ]:
ligandmpnn = LigandMPNN()

To run ligandmpnn on our poses, we have to use the .run() function. All tools and metrics should have this function. It is mandatory to provide a unique prefix for each run. Each score generated will be saved to the poses dataframe in the format prefix_scorename. The output files can be found in a folder called prefix in the working_directory set for the input poses. The .run() function always returns poses.

In [ ]:
my_poses = ligandmpnn.run(poses=my_poses, prefix='ligmpnn_local', nseq=2, model_type='protein_mpnn')
display(my_poses.df)

Notice how the poses dataframe has changed! It now contains all the poses generated from LigandMPNN and the corresponding scores. 

Since we did not provide a jobstarter when we set up ligandmpnn, it ran on the local machine, because it defaulted to the jobstarter we set when creating our poses. We can run LigandMPNN using another jobstarter, either by providing LigandMPNN a default jobstarter or calling the .run function with the jobstarter option.

Since we want to run LigandMPNN on the original poses again, we have to reset to the original input (this is in general not recommended, as it will delete the dataframe and might lead to reuse of prefix, reading in previously generated data).

In [ ]:
my_poses.reset_poses(force_reset_df=True)
my_poses = ligandmpnn.run(poses=my_poses, prefix='ligmpnn_cpu', nseq=2, model_type='protein_mpnn', jobstarter=slurm_cpu_jobstarter)
display(my_poses.df)

In [ ]:
my_poses.reset_poses(force_reset_df=True)
display(my_poses.df)

gpu_ligandmpnn = LigandMPNN(jobstarter=slurm_gpu_jobstarter)
my_poses = gpu_ligandmpnn.run(poses=my_poses, prefix='ligmpnn_gpu', nseq=2, model_type='protein_mpnn', jobstarter=slurm_gpu_jobstarter)
display(my_poses.df)

You can provide options to your runner. These will be applied to all input poses. If you want to apply pose-specific options, you can use pose_options. The pose_options option reads in either a dataframe column (where each row contains the option for the corresponding pose) or a list of options for each pose. You can combine options and pose_options freely. If setting an option using both the generic options and pose_options, the pose_options will overwrite the generic one (but this should in general be avoided to prevent confusion). Some runners have explicit arguments for options that are mandatory or used very often (like model_type for LigandMPNN).

The options must be formatted in the same way as when running them directly. E.g. LigandMPNN uses options and flags with a -- prefix, whereas Rosetta uses options and flags indicated by -.

We are going to set general options to slightly bias ProteinMPNN to prefer Ala residues and provide a seed (to generate reproducible data, not recommended for general runs) and provide pose_options to preserve certain residues which should not be designed.

In [ ]:
my_poses.reset_poses(force_reset_df=True)

pose_options = ["--fixed_residues 'A23 A69 AA126 A177'", "--fixed_residues 'A25 A73 A123 A177'", "--fixed_residues 'A26 A82 A119 A177'"]
my_poses.df['mpnn_pose_options'] = pose_options
print('Input poses with pose_options:')
display(my_poses.df[['poses', 'mpnn_pose_options']])

my_poses = ligandmpnn.run(poses=my_poses, prefix='options', nseq=10, model_type='protein_mpnn', options='--bias_AA A:0.1 --seed 111', pose_options='mpnn_pose_options')
print('Output poses:')
display(my_poses.df)

We can now predict the sequences generated by LigandMPNN by running ESMFold on our poses.

In [ ]:
from protflow.tools.esmfold import ESMFold
esmfold = ESMFold(jobstarter=slurm_gpu_jobstarter)
my_poses = esmfold.run(poses=my_poses, prefix='esm_pred')
display(my_poses.df)

We can now calculate the backbone RMSD of our poses with the new sequences compared to the original input using rmsd metrics. Metrics work in the same way as runners, but just calculate scores and (in general) do not manipulate the poses.

In [ ]:
import protflow.metrics.rmsd as rmsd_metrics
bb_rmsd_runner = rmsd_metrics.BackboneRMSD(ref_col='input_poses', atoms=['CA'], jobstarter=local_jobstarter)
my_poses = bb_rmsd_runner.run(poses=my_poses, prefix='CA_rmsd', overwrite=False)
display(my_poses.df)